# Chapter 5. Hadoop I/O<br>
## 5.1 데이터 무결성<br>
- I/O란? 입력(Input)/출력(Output)의 약자로, 컴퓨터 및 주변장치에 대하여 데이터를 전송하는 프로그램, 운영 혹은 장치를 일컫는 말<br><br>
- 무결성이란? 데이터베이스에 저장된 데이터 값과 그것이 표현하는 현실 세계의 실제값이 일치하는 정확성을 의미한다<br><br>
- 디스크나 네트워크상의 모든 I/O 조작은 데이터를 읽거나 쓸 때, 에러가 발생할 수 있기 때문에 커다란 데이터의 경우 데이터가 손상될 가능성도 그만큼 커진다.

### 5.1.1 HDFS의 데이터 무결성<br>
- 체크섬(checksum)이란? 중복 검사의 한 형태로, 오류 정정을 통해, 공간(전자 통신)이나 시간(기억 장치) 속에서 송신된 자료의 무결성을 보호하는 단순한 방법
    - 체크섬 원리 : https://m.blog.naver.com/PostView.nhn?blogId=dkwltmdgus&logNo=220537080579&proxyReferer=https:%2F%2Fwww.google.com%2F<br><br>
- write : 데이터노드는 데이터와 체크섬을 저장하기 전에 수신한 데이터를 검증할 책임이 있다.
    - 클라이언트의 수신 데이터 or 복제과정에서 다른 데이터노드로부터 수신한 데이터 모두 해당<br><br>
- read : 클라이언트 역시 데이터노드에 저장된 체크섬과 수신된 데이터로부터 계산된 체크섬을 검증.
    - 각 데이터노드는 체크섬 검증 로그를 영구 저장하기 때문에 각각의 블록이 검증되었던 마지막 시간을 알고 있다.


### 5.1.2 LocalFileSystem<br>
- 체크섬은 전체 계산 성능에 미치는 영향이 미미함.<br><br>
- 그러나 기존 FS이 자체적으로 체크섬을 지원한다면 LocalFileSystem의 **체크섬을 비활성화할 수 있다.**<br><br>
- 이를 위해, LocalFileSystem --> RawLocalFileSystem 로 대신 사용

### 5.1.3 ChecksumFileSystem<br>
**pass! 몰라도 될 듯**

## 5.2 압축<br>

- 파일 압축은 파일 저장 공간을 줄이고, 네트워크 또는 디스크로부터데이터 전송을 고속화할 수 있다.<br><br>
___
압축 포멧 | 도구 | 알고리즘 | 파일 확장명 | 분할 가능
-|-|-|-|-
DEFLATE | N/A | DEFLATE | .deflate | No
gzip | gzip | DEFLATE | .gz | No
bzip2 | bzip2 | bzip2 | .bz2 | **Yes**
LZO | lzop | LZO | .lzo | No
LZ4 | N/A | LZ4 | .lz4 | No
Snappy | N/a | Snappy | .snappy | No

    모든 압축 알고리즘은 압축과 해제가 빠를수록 공간이 늘어나는 희생을 감수해야하기 때문에 공간과 시간은 trade-off 관계이다.

- 분할가능한 압축 포멧은 맵리듀스에 적합하다.<br>



### 5.2.1 코덱<br>

- 코덱은 압축-해제 알고리즘을 구현한 것이고, 하둡에서 코덱은 CompressionCodec 인터페이스로 구현된다.
<br><br>
___

#### <br>CompressionCodec을 통한 압축 해제 스트림<br>

- CompressionCodec은 데이터를 쉽게 압축하거나 해제해주는 두 개의 메서드를 제공한다.<br>
- 나머지는 굳이 이해하지 않아도 될 것 같고, 심심하면 개인 공부로<br><br>

메서드 및 클래스 | 내용
-|-
createOutputStream 메서드 | 출력 스트림에 쓸 데이터 압축하여 CompressionOutputStream 생성
createInputStream 메서드 | 입력 스트림으로부터 읽어 들인 데이터를 압축 해제하여 CompressionInputStream 반환
c\compressionOutputStream | 기존의 java.util.zip.DeflaterOutputStream과 비슷
CompressionInputStream |  기존의 java.util.zip.DeflaterInputStream과 비슷



## 5.3 직렬화<br>

- 직렬화, serialization : 네트워크 전송을 위해 객체 -> byte Stream 으로 전환하는 과정
- 역직렬화, deserializtion : byte Stream -> 객체로 역전환하는 과정
___
- 직렬화는 프로세스 간 통신과 영속적인 저장소와 같은 분산 데이터 처리의 독특한 두 영역에서 나타난다.
- 하둡시스템에서 노드 사이의 프로세스 간 통신은 RPC, Remote Procedure Call, 원격 프로시저 호출을 사용한다.
- RPC 프로토콜은 원격 노드로 보내기 위한 메세지를 하나의 binary stream으로 구성하기 위해 직렬화를 사용하고, 그 후 원격 노드에서 binary stream을 원본 메세지로 재구성하기 위해 역직렬화를 사용한다.
___
**RPC 직렬화 포맷이 유익한 이유**

Pros | Desc
-|-
**간결성** | 간결한 포맷을 사용하면 데이터 센터에서 가장 희소성이 높은 자원인 네트워크 대역폭을 절약할 수 있다.
**고속화** | 프로세스 간 통신은 분산 시스템을 위한 `백본(backbone)`을 형성하기 때문에 직렬화와 역직렬화는 가능한 오버헤드가 작아야한다.
**확장성** | 새로운 요구사항을 만족시키기 위해 점차 변경되므로 클라이언트와 서버 사이의 통제 방식과 관련된 프로토콜의 발전도 직관적이어야 한다.(확장성이 높아야 한다.)
**상호운용성** | 일부 시스템을 위해 다양한 언어로 작성된 클라이언트를 지원하는 편이 좋으며, 이를 가능하도록 포멧을 설계할 필요가 있다.

    요약 : 저장소 포맷은 간결하고(저장공간의 효율), 빠르고(오버헤드를 최소화), 확장 가능하고(예전 포맷), 상호운용(다양한 언어 사용)할 수 있어야한다.

`백본`은 자신에게 연결되어 있는 소형 회선들로부터 데이터를 모아 빠르게 전송할 수 있는 대규모 전송회선을 말한다.
<br><br>
___
<br>

*하둡은 Writable이라는 직렬화 포맷을 사용하지만, Java 이외의 다른 언어를 사용하는 것은 어렵다.*


### 5.3.1 Writable 인터페이스<br>

Writable 인터페이스는 자신의 상태정보를 DataOutput Binary Stream으로 쓰기 위한 메서드와 DataInput Binary Stream으로부터 상태 정보를 읽기 위한 메서드를 정의한다.

    직렬화와 역직렬화를 위한 내용. 이정도만 알고 넘어가자! 

### 5.3.2 Writable 클래스<br>

- 인터페이스와 클래스(추상클래스)의 차이 : https://brunch.co.kr/@kd4/6
- 이런 자원(인터페이스, 클래스)들을 이용해서 어떻게 직렬화를 이루는지 설명하고 있다.
- 그 이상의 설명은 생략

### 5.3.3 커스텀 Writable 구현하기<br>

- 하둡은 대부분의 목적을 만족시키는 유용한 Writable 구현체를 제공한다.
- 하지만 보다 커스텀하게 하기 위해서는 직접 구현할 필요가 있다.

### 5.3.4 직렬화 프레임워크<br>

대부분의 맵리듀스 프로그램이 Writable 키와 값 타입을 사용한다.

## 5.4 파일 기반 데이터 구조<br>

맵리듀스 기반의 데이터 처리를 위해 바이너리 데이터의 각 BLOB을 한 파일에 몽땅 담아두는 것은 확장성에 좋지 않다. 따라서 하둡은 이를 위해 고수준 컨테이너들을 개발했다.

### 5.4.1 SequenceFile<br>

- 로그 파일을 생각해보면, Binary 형태로 로그를 남기고 싶다면 일반 텍스트 포맷은 적합하지 않다.
    - 하둡의 SequenceFile 클래스는 Binary key-value 쌍에 대한 영속적인 데이터 구조를 제공하기 때문에 이러한 상황에 딱 맞는 구조라 할 수 있다.